In [1]:
import neuronencodings as ne
from meshparty import mesh_io

# Setting up

In [2]:
d = ne.data.fetch_dset_dirs("pinky100_orphan_dends")[0]
fs = ne.data.utils.files_from_dir(d)

In [3]:
meshmeta = mesh_io.MeshMeta()

In [4]:
mesh = meshmeta.mesh(fs[0])

In [5]:
chkpt_fname = "/usr/people/nturner/seungmount/research/nick_and_sven/models_nick/p100_dend_outer/packaged/model_180000.chkpt"
model = ne.utils.load_autoencoder("PointNetAE", n_pts=1000, bottle_fs=64, chkpt_fname=chkpt_fname)

## Picking cells

In [6]:
import random
random.seed(12345)
files = random.sample(fs, 300)

In [7]:
INF_DIR = "/usr/people/nturner/seungmount/research/nick_and_sven/models_nick/p100_dend_outer/inference"

In [8]:
with open(f"{INF_DIR}/180928_cells.csv", "w+") as f:
    f.write("\n".join(files))

In [9]:
import os
cell_id = lambda f : int(os.path.basename(f.split(".")[0]))

In [10]:
output_fname = lambda f: f"{INF_DIR}/{cell_id(f)}.h5"

In [11]:
output_fname(files[0])

'/usr/people/nturner/seungmount/research/nick_and_sven/models_nick/p100_dend_outer/inference/546254140358.h5'

In [12]:
import h5py

In [13]:
def write_h5(vectors, centers, fname):
    with h5py.File(fname) as f:
        f.create_dataset("vertices", data=vectors)
        f.create_dataset("centers", data=centers)

# Running inference

In [14]:
meshes = list(meshmeta.mesh(f) for f in files)

In [15]:
all_vecs, all_center_inds = ne.pred.encode_meshs_by_views(model, meshes, 1000, pc_align=True, order="sequential")

In [17]:
for (f, v, c, m) in zip(files, all_vecs, all_center_inds, meshes):
    cs = m.vertices[c,:]
    write_h5(v, cs, output_fname(f))